# Objective

To create a simple model that can predict which medication from a given list
would be appropriate for a patient based on their symptoms/treatment goals.

In [ ]:
!pip install pandas
!pip install datasets

In [1]:
import os
import re
import glob
import shutil
import string
import pathlib


data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','data'))


os.environ['MPLCONFIGDIR'] = os.path.join(data_dir,'plt_configs')
import matplotlib.pyplot as plt


os.environ['HF_HOME'] = os.path.join(data_dir,'hf_cache')
import datasets

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.layers import TextVectorization

# Data collection

Drug review dataset obtained from Hugging Face

In [2]:
dataset = datasets.load_dataset("flxclxc/encoded_drug_reviews")

Using custom data configuration flxclxc--encoded_drug_reviews-ee0cdba36988e67d
Found cached dataset json (/tf/data/hf_cache/datasets/flxclxc___json/flxclxc--encoded_drug_reviews-ee0cdba36988e67d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
df = pd.DataFrame(dataset)

In [4]:
print(df)

                                                   train
0      {'patient_id': 184648, 'drugName': 'Efudex', '...
1      {'patient_id': 25268, 'drugName': 'Flector Pat...
2      {'patient_id': 172019, 'drugName': 'Amitiza', ...
3      {'patient_id': 196063, 'drugName': 'Stendra', ...
4      {'patient_id': 225264, 'drugName': 'Bupropion'...
...                                                  ...
53466  {'patient_id': 199190, 'drugName': 'Depo-Prove...
53467  {'patient_id': 188476, 'drugName': 'ParaGard',...
53468  {'patient_id': 105752, 'drugName': 'Methylpred...
53469  {'patient_id': 56713, 'drugName': 'Meclizine',...
53470  {'patient_id': 215006, 'drugName': 'Fluoxetine...

[53471 rows x 1 columns]


In [5]:
complete_dataset = dataset['train'].to_pandas()
complete_dataset.head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,encoded
0,184648,Efudex,basal cell carcinoma,"""I have BCC on my upper arm and SCC on upper l...",1.0,"August 30, 2016",16,36,"[-0.0633561835, 0.0115883639, -0.0027463636, 0..."
1,25268,Flector Patch,pain,"""I tore my shoulder labrum and the pain can be...",8.0,"May 29, 2014",40,45,"[-0.083280459, 0.0182377025, 0.0619471855, 0.0..."
2,172019,Amitiza,irritable bowel syndrome,"""Amitiza is the best if you have ibs!""",10.0,"July 13, 2016",9,8,"[-0.0300639421, -0.0081300493, 0.0343461707, 0..."
3,196063,Stendra,erectile dysfunction,"""Viagra works in a strong, crude way with side...",10.0,"November 10, 2014",82,141,"[-0.0037669495, -0.0845683292, 0.0196341239, 0..."
4,225264,Bupropion,depression,"""I really wanted Wellbutrin to work. I was giv...",3.0,"October 4, 2015",15,62,"[-0.0633124188, 0.0167291258, 0.0707527027, 0...."


In [6]:
print(df.info()) # 53,471 total drug reviews in dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53471 entries, 0 to 53470
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   53471 non-null  object
dtypes: object(1)
memory usage: 417.9+ KB
None


In [7]:
complete_dataset.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53471 entries, 0 to 53470
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patient_id     53471 non-null  int64  
 1   drugName       53471 non-null  object 
 2   condition      53471 non-null  object 
 3   review         53471 non-null  object 
 4   rating         53471 non-null  float64
 5   date           53471 non-null  object 
 6   usefulCount    53471 non-null  int64  
 7   review_length  53471 non-null  int64  
 8   encoded        53471 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 3.7+ MB


# Data Cleaning

-Isolated drugs from the dataset

In [8]:
# unique values in column "drugName"
drugs = complete_dataset['drugName'].unique()
for drug in drugs:
    print(drug)

Efudex
Flector Patch
Amitiza
Stendra
Bupropion
Amiodarone
Sprintec
Acetaminophen / tramadol
Sertraline
Lisinopril
Levonorgestrel
Liraglutide
Fluoxetine
Methylprednisolone
Doxylamine / pyridoxine
Zoladex
Clarithromycin
Tranylcypromine
Phentermine
Prednisolone
Tarceva
Ezetimibe / simvastatin
Belsomra
Enbrel
Flexeril
Ethinyl estradiol / levonorgestrel
Prozac
Clomipramine
Klonopin
Ethinyl estradiol / norgestimate
Zovirax
Adipex-P
Vilazodone
Etonogestrel
Buspirone
Phentermine / topiramate
Micardis HCT
Dapsone
TriNessa
Estarylla
Venlafaxine
Aviane
Geodon
Tiotropium
Remicade
Synthroid
Minocycline
Ativan
Eletriptan
Levoxyl
Duloxetine
Dexbrompheniramine / pseudoephedrine
Loestrin 24 Fe
Haloperidol
Nexplanon
Robaxin
Cefdinir
Ortho Tri-Cyclen Lo
Desogen
Hiprex
Lorazepam
Loratadine / pseudoephedrine
Lyrica
Norco
Ethinyl estradiol / norelgestromin
Ambrisentan
Trulicity
Duac
Zyrtec
Propranolol
Alli
Exemestane
Mirena
NuvaRing
Ethinyl estradiol / norethindrone
Desogestrel / ethinyl estradiol
Magnesium

-Selected drugs with a review count > 200 reviews

In [9]:
pd.set_option("display.max_rows", None)
frequence = complete_dataset['drugName'].value_counts()
print(frequence)

Levonorgestrel                                                                                      1265
Etonogestrel                                                                                        1081
Ethinyl estradiol / norethindrone                                                                    869
Nexplanon                                                                                            736
Ethinyl estradiol / norgestimate                                                                     649
Ethinyl estradiol / levonorgestrel                                                                   591
Phentermine                                                                                          539
Sertraline                                                                                           506
Escitalopram                                                                                         452
Mirena                                                 

-Removed brand name drugs from dataset so that model did not classify brand/generic as two separate medications 

ex. Lexapro/Escitalopram or Chantix/Varenicline

In [10]:
base_dir = os.path.join(data_dir,'drugs')
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

drug_names = ["Levonorgestrel",
              "Etonogestrel",
              "Ethinyl estradiol / norethindrone",
              "Ethinyl estradiol / norgestimate",
              "Ethinyl estradiol / levonorgestrel",
              "Phentermine",
              "Sertraline",
              "Escitalopram",
              "Mirena",
              "Gabapentin",
              "Miconazole",
              "Bupropion",
              "Venlafaxine",
              "Duloxetine",
              "Tramadol",
              "Clonazepam",
              "Citalopram",
              "Medroxyprogesterone",
              "Bupropion / naltrexone",
              "Varenicline",
              "Metronidazole",
              "Drospirenone / ethinyl estradiol",
              "Tioconazole",
              "Depo-Provera",
              "Liraglutide",
              "Fluoxetine",
              "Quetiapine",
              "Lo Loestrin Fe",
              "Alprazolam",
              "Amitriptyline",
              "Doxycycline",
              "Desvenlafaxine",
              "Trazodone",
              "Suprep Bowel Prep Kit",
              "Paroxetine",
              "Bisacodyl",
              "Lorcaserin"]
drug_directories = []

for drug_name in drug_names:
    current_drug_dir = os.path.join(data_dir,'drugs',drug_name)
    print(current_drug_dir)
    drug_directories.append(current_drug_dir)
    if not os.path.exists(current_drug_dir):
        os.makedirs(current_drug_dir)


/tf/data/drugs/Levonorgestrel
/tf/data/drugs/Etonogestrel
/tf/data/drugs/Ethinyl estradiol / norethindrone
/tf/data/drugs/Ethinyl estradiol / norgestimate
/tf/data/drugs/Ethinyl estradiol / levonorgestrel
/tf/data/drugs/Phentermine
/tf/data/drugs/Sertraline
/tf/data/drugs/Escitalopram
/tf/data/drugs/Mirena
/tf/data/drugs/Gabapentin
/tf/data/drugs/Miconazole
/tf/data/drugs/Bupropion
/tf/data/drugs/Venlafaxine
/tf/data/drugs/Duloxetine
/tf/data/drugs/Tramadol
/tf/data/drugs/Clonazepam
/tf/data/drugs/Citalopram
/tf/data/drugs/Medroxyprogesterone
/tf/data/drugs/Bupropion / naltrexone
/tf/data/drugs/Varenicline
/tf/data/drugs/Metronidazole
/tf/data/drugs/Drospirenone / ethinyl estradiol
/tf/data/drugs/Tioconazole
/tf/data/drugs/Depo-Provera
/tf/data/drugs/Liraglutide
/tf/data/drugs/Fluoxetine
/tf/data/drugs/Quetiapine
/tf/data/drugs/Lo Loestrin Fe
/tf/data/drugs/Alprazolam
/tf/data/drugs/Amitriptyline
/tf/data/drugs/Doxycycline
/tf/data/drugs/Desvenlafaxine
/tf/data/drugs/Trazodone
/tf/data

In [11]:
data = np.array(["Levonorgestrel",
                 "Etonogestrel",
                 "Ethinyl estradiol / norethindrone",
                 "Ethinyl estradiol / norgestimate",
                 "Ethinyl estradiol / levonorgestrel",
                 "Phentermine",
                 "Sertraline",
                 "Escitalopram",
                 "Mirena",
                 "Gabapentin",
                 "Miconazole",
                 "Bupropion",
                 "Venlafaxine",
                 "Duloxetine",
                 "Tramadol",
                 "Clonazepam",
                 "Citalopram",
                 "Medroxyprogesterone",
                 "Bupropion / naltrexone",
                 "Varenicline",
                 "Metronidazole",
                 "Drospirenone / ethinyl estradiol",
                 "Tioconazole",
                 "Depo-Provera",
                 "Liraglutide",
                 "Fluoxetine",
                 "Quetiapine",
                 "Lo Loestrin Fe",
                 "Alprazolam",
                 "Amitriptyline",
                 "Doxycycline",
                 "Desvenlafaxine",
                 "Trazodone",
                 "Suprep Bowel Prep Kit",
                 "Paroxetine",
                 "Bisacodyl",
                 "Lorcaserin"])
s = pd.Series(data)

In [12]:
print(s[:])

0                         Levonorgestrel
1                           Etonogestrel
2      Ethinyl estradiol / norethindrone
3       Ethinyl estradiol / norgestimate
4     Ethinyl estradiol / levonorgestrel
5                            Phentermine
6                             Sertraline
7                           Escitalopram
8                                 Mirena
9                             Gabapentin
10                            Miconazole
11                             Bupropion
12                           Venlafaxine
13                            Duloxetine
14                              Tramadol
15                            Clonazepam
16                            Citalopram
17                   Medroxyprogesterone
18                Bupropion / naltrexone
19                           Varenicline
20                         Metronidazole
21      Drospirenone / ethinyl estradiol
22                           Tioconazole
23                          Depo-Provera
24              

In [13]:
drug_datasets = []
for drug_name in drug_names:

    filtered_df = complete_dataset[complete_dataset['drugName'] == drug_name]
    drug_datasets.append(filtered_df)

In [14]:
drug_names
drug_directories
drug_datasets

for i in range(len(drug_names)):
    review_counter = 0
    for text in drug_datasets[i]['review']:
        with open(os.path.join(drug_directories[i],str(review_counter)+'.txt'), 'w') as f:
            f.write(text)
        review_counter+=1

# Training model using the dataset

In [15]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    pathlib.Path(base_dir),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 13923 files belonging to 35 classes.
Using 11139 files for training.


In [16]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print("Patient Review: ", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])

Patient Review:  b'"This is so awesome reading everyone\'s reviews. These are real reviews to me. You read other diet adds, they are not real. I started in late August, 28 2013. I have lost a total of 15lbs so far! Looking to lose more. I was near the 200 lb mark. It\'s been like that since I had my last child in 2005. All this time I had been working out, I saw no changes. I needed a change because I have insulin resistance. That is scary to me. Diabetes runs in my family both parents. I love this pill!. It\'s amazing how much energy I feel. I did feel jittery when taking at first and not feel hungry but I ate anyways. I watch what I eat. Insomnia and dry mouth. Drink lots of water. I just want to feel better about myself. "'
Label: 26
Patient Review:  b'"Gabapentin is a miracle worker. While I was suffering with depression and severe anxiety, I was against taking any type of medications to help me however things were starting to get out control. I am very happy with how things are no

In [17]:
for i, label in enumerate(raw_train_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to Alprazolam
Label 1 corresponds to Amitriptyline
Label 2 corresponds to Bisacodyl
Label 3 corresponds to Bupropion
Label 4 corresponds to Bupropion 
Label 5 corresponds to Citalopram
Label 6 corresponds to Clonazepam
Label 7 corresponds to Depo-Provera
Label 8 corresponds to Desvenlafaxine
Label 9 corresponds to Doxycycline
Label 10 corresponds to Drospirenone 
Label 11 corresponds to Duloxetine
Label 12 corresponds to Escitalopram
Label 13 corresponds to Ethinyl estradiol 
Label 14 corresponds to Etonogestrel
Label 15 corresponds to Fluoxetine
Label 16 corresponds to Gabapentin
Label 17 corresponds to Levonorgestrel
Label 18 corresponds to Liraglutide
Label 19 corresponds to Lo Loestrin Fe
Label 20 corresponds to Lorcaserin
Label 21 corresponds to Medroxyprogesterone
Label 22 corresponds to Metronidazole
Label 23 corresponds to Miconazole
Label 24 corresponds to Mirena
Label 25 corresponds to Paroxetine
Label 26 corresponds to Phentermine
Label 27 corresponds to 

# Create validation set

In [18]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    pathlib.Path(base_dir),
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)


Found 13923 files belonging to 35 classes.
Using 2784 files for validation.


# Prepare dataset for training

-Utilized TensorFlow Text Vectorization

In [19]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [20]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [21]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [22]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [23]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'"I\'m on Mylans generic version of Generess Fe and so far its been okay. I\'ve been on 4 different birth controls in the past year and a half and I was put on this one to try to eliminate a cyst I\'ve had since November. While I can say the cyst pain had gone away I have now noticed that I get extremely nauseous on this pill. I\'m sure it\'s just my body adjusting but I feel terrible. I take the pill at 9 every night so I think I have to make sure that I don\'t have an empty stomach when I take it. This pill has also made me break out a bit but hopefully that will go away in the next few months."', shape=(), dtype=string)
Label Ethinyl estradiol 
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  25,   13, 8084,  575, 1488,   12, 2842,  571,    3,   22,  152,
          43,   24,  659,   33,   24,   13,  139,  248,   90, 1117,   17,
           4,  304,  122,    3,    8,  272,    3,    2,   10,  156,   13,
          14,   79,    5,  190,    5,

In [24]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  contraceptive
 313 --->  8
Vocabulary size: 10000


In [25]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [26]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Create the model

Loss function: Sparse Categorical Crossentropy (multi classification)

Optimizer: nadam (Nesterov-accelerated Adaptive Moment Estimation)

Activation: Softmax

In [27]:
embedding_dim = 16

In [28]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(len(drug_names), activation=tf.nn.softmax)])


In [29]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer='nadam',
              metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 37)                629       
                                                                 
Total params: 160,645
Trainable params: 160,645
Non-trainable params: 0
__________________________________________________

# Train the model

In [31]:
epochs = 300
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/300
349/349 [==============================] - 2s 2ms/step - loss: 3.4619 - accuracy: 0.1397 - val_loss: 3.3173 - val_accuracy: 0.1469
Epoch 2/300
349/349 [==============================] - 1s 2ms/step - loss: 3.2896 - accuracy: 0.1526 - val_loss: 3.2786 - val_accuracy: 0.1469
Epoch 3/300
349/349 [==============================] - 1s 2ms/step - loss: 3.2521 - accuracy: 0.1528 - val_loss: 3.2443 - val_accuracy: 0.1469
Epoch 4/300
349/349 [==============================] - 1s 2ms/step - loss: 3.2106 - accuracy: 0.1530 - val_loss: 3.2015 - val_accuracy: 0.1476
Epoch 5/300
349/349 [==============================] - 1s 2ms/step - loss: 3.1638 - accuracy: 0.1576 - val_loss: 3.1504 - val_accuracy: 0.1501
Epoch 6/300
349/349 [==============================] - 1s 2ms/step - loss: 3.1060 - accuracy: 0.1679 - val_loss: 3.0931 - val_accuracy: 0.1677
Epoch 7/300
349/349 [==============================] - 1s 2ms/step - loss: 3.0460 - accuracy: 0.1839 - val_loss: 3.0306 - val_accuracy: 0.1814

Epoch 58/300
349/349 [==============================] - 1s 2ms/step - loss: 1.4506 - accuracy: 0.5577 - val_loss: 1.4774 - val_accuracy: 0.5697
Epoch 59/300
349/349 [==============================] - 1s 2ms/step - loss: 1.4433 - accuracy: 0.5579 - val_loss: 1.4676 - val_accuracy: 0.5740
Epoch 60/300
349/349 [==============================] - 1s 2ms/step - loss: 1.4316 - accuracy: 0.5636 - val_loss: 1.4582 - val_accuracy: 0.5761
Epoch 61/300
349/349 [==============================] - 1s 2ms/step - loss: 1.4185 - accuracy: 0.5716 - val_loss: 1.4483 - val_accuracy: 0.5783
Epoch 62/300
349/349 [==============================] - 1s 2ms/step - loss: 1.4105 - accuracy: 0.5720 - val_loss: 1.4390 - val_accuracy: 0.5797
Epoch 63/300
349/349 [==============================] - 1s 2ms/step - loss: 1.3999 - accuracy: 0.5776 - val_loss: 1.4299 - val_accuracy: 0.5841
Epoch 64/300
349/349 [==============================] - 1s 2ms/step - loss: 1.3871 - accuracy: 0.5786 - val_loss: 1.4212 - val_accuracy:

Epoch 115/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0305 - accuracy: 0.6807 - val_loss: 1.1387 - val_accuracy: 0.6548
Epoch 116/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0213 - accuracy: 0.6863 - val_loss: 1.1357 - val_accuracy: 0.6530
Epoch 117/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0188 - accuracy: 0.6911 - val_loss: 1.1317 - val_accuracy: 0.6552
Epoch 118/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0192 - accuracy: 0.6836 - val_loss: 1.1298 - val_accuracy: 0.6537
Epoch 119/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0084 - accuracy: 0.6835 - val_loss: 1.1253 - val_accuracy: 0.6573
Epoch 120/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0061 - accuracy: 0.6893 - val_loss: 1.1231 - val_accuracy: 0.6545
Epoch 121/300
349/349 [==============================] - 1s 2ms/step - loss: 1.0005 - accuracy: 0.6917 - val_loss: 1.1200 - val_ac

349/349 [==============================] - 1s 2ms/step - loss: 0.6554 - accuracy: 0.7934 - val_loss: 0.9308 - val_accuracy: 0.6994
Epoch 228/300
349/349 [==============================] - 1s 2ms/step - loss: 0.6556 - accuracy: 0.7931 - val_loss: 0.9300 - val_accuracy: 0.7001
Epoch 229/300
349/349 [==============================] - 1s 2ms/step - loss: 0.6536 - accuracy: 0.7938 - val_loss: 0.9273 - val_accuracy: 0.6994
Epoch 230/300
349/349 [==============================] - 1s 2ms/step - loss: 0.6530 - accuracy: 0.7961 - val_loss: 0.9274 - val_accuracy: 0.6986
Epoch 231/300
349/349 [==============================] - 1s 2ms/step - loss: 0.6472 - accuracy: 0.7978 - val_loss: 0.9261 - val_accuracy: 0.7004
Epoch 232/300
349/349 [==============================] - 1s 2ms/step - loss: 0.6429 - accuracy: 0.8013 - val_loss: 0.9253 - val_accuracy: 0.7001
Epoch 233/300
349/349 [==============================] - 1s 2ms/step - loss: 0.6435 - accuracy: 0.7977 - val_loss: 0.9248 - val_accuracy: 0.7008

In [32]:
loss, accuracy = model.evaluate(train_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

349/349 [==============================] - 0s 605us/step - loss: 0.3805 - accuracy: 0.8928
Loss:  0.38051557540893555
Accuracy:  0.8928090333938599


In [33]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

87/87 [==============================] - 0s 602us/step - loss: 0.8987 - accuracy: 0.7108
Loss:  0.8986846208572388
Accuracy:  0.7108476758003235


# Export the model

In [34]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('softmax')
])

export_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='nadam',
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)


# Inference on new data

In [40]:
examples = [
  "I need constipation relief. I haven't had a bowel movement in days.",
  "I need to quit smoking. I've been trying to quit for 5 years but haven't been able to stop.",
  "I am clinically obese and my doctor recommends that I lose some weight."
]

predictions=export_model.predict(examples)


1/1 [==============================] - 0s 21ms/step


In [41]:
for j in range(len(examples)):
    print(examples[j])
    prediction=predictions[j]
    for i, label in enumerate(raw_train_ds.class_names):
        print(label+':'+str(prediction[i]))

I need constipation relief. I haven't had a bowel movement in days.
Alprazolam:0.027992737
Amitriptyline:0.028081547
Bisacodyl:0.039304838
Bupropion:0.026375666
Bupropion :0.026448157
Citalopram:0.026295884
Clonazepam:0.026724856
Depo-Provera:0.026290558
Desvenlafaxine:0.026602026
Doxycycline:0.02651452
Drospirenone :0.026258783
Duloxetine:0.026896669
Escitalopram:0.026440028
Ethinyl estradiol :0.026290426
Etonogestrel:0.026256127
Fluoxetine:0.026316678
Gabapentin:0.026647054
Levonorgestrel:0.026720267
Liraglutide:0.0263603
Lo Loestrin Fe:0.026253564
Lorcaserin:0.026412234
Medroxyprogesterone:0.026308527
Metronidazole:0.027736874
Miconazole:0.026398908
Mirena:0.02646992
Paroxetine:0.0263931
Phentermine:0.026704764
Quetiapine:0.026496897
Sertraline:0.026322342
Suprep Bowel Prep Kit:0.026763124
Tioconazole:0.027407339
Tramadol:0.028944781
Trazodone:0.027157381
Varenicline:0.026279496
Venlafaxine:0.026651198
I need to quit smoking. I've been trying to quit for 5 years but haven't been abl

# Constipation relief 

Model predicts Bisacodyl:0.039304838

Bisacodyl is a laxative. Good prediction!

# Smoking cessation 

Model predicts Bupropion:0.033370513 and Varenicline:0.04371334

Bupropion and Varenicline (Brand name Chantix) are used for smoking cessation.

# Weight loss

Model predicts Phentermine:0.02896901 and Lorcaserin:0.030372752

Phentermine and Locaserin are prescribed for weight loss, so these are good choices. However, I would have also expected the model to predict Bupropion / Naltrexone (0.027201299) and Liraglutide (0.026953785 with a higher degree of confidence as these are also used to treat obesity. 